In [46]:
import folium
import geemap
from drought.data.pipeline import get_gpd_polygons, get_ee_polygons
import ee
from drought.data.ee_converter import gdf_to_ee_polygon
import shapely
import branca

In [9]:
ee.Initialize()

In [18]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [114]:
def viz_single_polygon(img: ee.Image, polygon_index: int,  band: str):
    polygon = get_gpd_polygons().geometry[polygon_index]
    region = gdf_to_ee_polygon(polygon)
    img_stats = geemap.image_stats(img.clip(region), scale=5000).getInfo()

    band_min = img_stats['min'][band]
    band_max = img_stats['max'][band]
    if band_min == band_max:
        band_max = band_min + 1
    colors = ['#ffff00', '#800026']

    lon, lat = polygon.centroid.x, polygon.centroid.y
    my_map = folium.Map(location=[lat, lon], zoom_start=9, control_scale=True)
    my_map.add_ee_layer(img.select(band).clip(region), 
    {"min": band_min, "max": band_max, 'palette': colors}, name=band)

    # Add legend
    colormap = branca.colormap.LinearColormap(colors=['yellow', 'red']).scale(band_min, band_max)
    colormap.add_to(my_map)
    display(my_map)

In [112]:
# ET and PET data for the first 6 months of 2019.
start_date = ee.Date('2019-01-01')
end_date = ee.Date('2019-01-06')\

ic = ee.ImageCollection('MODIS/006/MOD16A2') \
    .select(['ET', 'PET']) \
    .filterDate(start_date, end_date)


In [115]:
# Visualize a count of how many shots there are per pixel.
viz_single_polygon(ic.count(), polygon_index=7, band="ET")

In [119]:
# Visualize values.
viz_single_polygon(ic.first(), polygon_index=7, band="ET")

In [131]:
def viz_all_polygons(img: ee.Image,  band: str):
    polygons = get_gpd_polygons().geometry
    lon, lat = polygons.centroid.x.median(), polygons.centroid.y.median()
    my_map = folium.Map(location=[lat, lon], zoom_start=6, control_scale=True)

    for polygon in polygons:
        region = gdf_to_ee_polygon(polygon)
        img_stats = geemap.image_stats(img.clip(region), scale=5000).getInfo()

        band_min = img_stats['min'][band]
        band_max = img_stats['max'][band]
        if band_min == band_max:
            band_max = band_min + 1
        colors = ['#ffff00', '#800026']

        my_map.add_ee_layer(img.select(band).clip(region), 
        {"min": band_min, "max": band_max, 'palette': colors}, name=band)

    display(my_map)

In [132]:
viz_all_polygons(ic.first(), band="ET")

/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_68106/1774311182.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = polygons.centroid.x.median(), polygons.centroid.y.median()
/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_68106/1774311182.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = polygons.centroid.x.median(), polygons.centroid.y.median()


In [126]:
polygons = get_gpd_polygons().geometry
lon, lat = polygons.centroid.x.mean(), polygons.centroid.y.mean()
lon, lat

/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_68106/2393906390.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = polygons.centroid.x.mean(), polygons.centroid.y.mean()
/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_68106/2393906390.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = polygons.centroid.x.mean(), polygons.centroid.y.mean()


(-55.611509909353074, -6.676149526629572)